# Final Project
## Machine Learning: Prediction on bike sharing demand

In [6]:
# Import

import folium
from folium.plugins import MarkerCluster

# data set
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning
from tensorflow.keras import layers

### Importing the merged dataset

In [7]:
dataset = pd.read_csv('Joint_dataframe.csv')

In [8]:
dataset

,index,timestamp,In,Out,Station Id,Total,is_holiday,is_weekend,avg_temp,avg_temp_feel,avg_humidity,avg_windSpeed,date,start_lon,start_lat,longitude,latitude
0,0,2016-04-01,17,22,1,-5.0,0.0,0.0,8.520833,7.895833,66.500000,11.166667,1.0,-0.109971,51.5292,-0.109971,51.5292
1,1,2016-04-01,42,31,2,11.0,0.0,0.0,8.520833,7.895833,66.500000,11.166667,1.0,-0.197574,51.4996,-0.197574,51.4996
2,2,2016-04-01,51,39,3,12.0,0.0,0.0,8.520833,7.895833,66.500000,11.166667,1.0,-0.084606,51.5213,-0.084606,51.5213
3,3,2016-04-01,44,37,4,7.0,0.0,0.0,8.520833,7.895833,66.500000,11.166667,1.0,-0.120974,51.5301,-0.120974,51.5301
4,4,2016-04-01,48,38,5,10.0,0.0,0.0,8.520833,7.895833,66.500000,11.166667,1.0,-0.156876,51.4931,-0.156876,51.4931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22758,22758,2016-05-08,1,0,0,0.0,0.0,24.0,19.250000,19.229167,57.145833,15.562500,8.0,0.000000,0.0000,0.000000,0.0000
22759,22759,2016-05-08,1,0,0,0.0,0.0,24.0,19.250000,19.229167,57.145833,15.562500,8.0,0.000000,0.0000,0.000000,0.0000
22760,22760,2016-05-09,1,0,0,0.0,0.0,0.0,17.708333,17.645833,65.729167,16.145833,9.0,0.000000,0.0000,0.000000,0.0000
22761,22761,2016-05-09,1,0,0,0.0,0.0,0.0,17.708333,17.645833,65.729167,16.145833,9.0,0.000000,0.0000,0.000000,0.0000


## LSTM (Long short-term memory neural networks)

In [9]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset.iloc[0:train_size], dataset.iloc[train_size:len(dataset)]
print(len(train),len(test))

18210 4553


In [10]:
from sklearn.preprocessing import RobustScaler

f_transformer = RobustScaler()
feature_columns = ['avg_temp', 'avg_temp_feel', 'avg_humidity', 'avg_windSpeed']
f_transformer = f_transformer.fit(train[feature_columns].to_numpy())
train.loc[:, feature_columns] = f_transformer.transform(train[feature_columns].to_numpy())

/Users/hchae/opt/anaconda3/envs/py3/lib/python3.10/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


In [12]:
Total_transformer = RobustScaler()
Total_transformer = Total_transformer.fit(train[['Total']])
train['Total'] = Total_transformer.transform(train[['Total']])
train.head()

/var/folders/2c/q0jzg_j92xjbxq26p0b0hmbh0000gn/T/ipykernel_24459/507813015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Total'] = Total_transformer.transform(train[['Total']])


,index,timestamp,In,Out,Station Id,Total,is_holiday,is_weekend,avg_temp,avg_temp_feel,avg_humidity,avg_windSpeed,date,start_lon,start_lat,longitude,latitude
0,0,2016-04-01,17,22,1,-0.161290,0.0,0.0,-0.333333,-0.111675,-0.07,-0.526946,1.0,-0.109971,51.5292,-0.109971,51.5292
1,1,2016-04-01,42,31,2,0.354839,0.0,0.0,-0.333333,-0.111675,-0.07,-0.526946,1.0,-0.197574,51.4996,-0.197574,51.4996
2,2,2016-04-01,51,39,3,0.387097,0.0,0.0,-0.333333,-0.111675,-0.07,-0.526946,1.0,-0.084606,51.5213,-0.084606,51.5213
3,3,2016-04-01,44,37,4,0.225806,0.0,0.0,-0.333333,-0.111675,-0.07,-0.526946,1.0,-0.120974,51.5301,-0.120974,51.5301
4,4,2016-04-01,48,38,5,0.322581,0.0,0.0,-0.333333,-0.111675,-0.07,-0.526946,1.0,-0.156876,51.4931,-0.156876,51.4931


In [13]:
test.loc[:, feature_columns] = f_transformer.transform(test[feature_columns].to_numpy())
test.head()

/Users/hchae/opt/anaconda3/envs/py3/lib/python3.10/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,index,timestamp,In,Out,Station Id,Total,is_holiday,is_weekend,avg_temp,avg_temp_feel,avg_humidity,avg_windSpeed,date,start_lon,start_lat,longitude,latitude
18210,18210,2016-04-25,22,21,169,1.0,0.0,0.0,-0.493056,-0.639594,0.855,0.494012,25.0,-0.165164,51.5147,-0.165164,51.5147
18211,18211,2016-04-25,56,33,170,23.0,0.0,0.0,-0.493056,-0.639594,0.855,0.494012,25.0,-0.108068,51.5278,-0.108068,51.5278
18212,18212,2016-04-25,310,21,171,289.0,0.0,0.0,-0.493056,-0.639594,0.855,0.494012,25.0,-0.186754,51.4916,-0.186754,51.4916
18213,18213,2016-04-25,27,19,172,8.0,0.0,0.0,-0.493056,-0.639594,0.855,0.494012,25.0,-0.173716,51.4912,-0.173716,51.4912
18214,18214,2016-04-25,32,44,173,-12.0,0.0,0.0,-0.493056,-0.639594,0.855,0.494012,25.0,-0.113001,51.5049,-0.113001,51.5049


In [14]:
test['Total'] = Total_transformer.transform(test[['Total']])
test['Total'].head()

/var/folders/2c/q0jzg_j92xjbxq26p0b0hmbh0000gn/T/ipykernel_24459/465734754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Total'] = Total_transformer.transform(test[['Total']])


18210      1.0
18211     23.0
18212    289.0
18213      8.0
18214    -12.0
Name: Total, dtype: float64

In [15]:
## Create X_train, y_train, X_test and y_test

def createDataset(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        data = X.iloc[i:(i + time_steps)].values
        Xs.append(data)        
        ys.append(y.iloc[i + time_steps])
        
    return np.array(Xs), np.array(ys)

time_steps = 10
X_train, y_train = createDataset(train, train.Total, time_steps)
X_test, y_test = createDataset(test, test.Total, time_steps)
print(X_train.shape, y_train.shape)

(18200, 10, 17) (18200,)


In [18]:
# Predicting Demand 

import keras

model = keras.Sequential()
model.add(
  layers.Bidirectional(
    layers.LSTM(
      units = 128, 
      input_shape = (X_train.shape[1], X_train.shape[2])
    )
  )
)
model.add(layers.Dropout(rate = 0.2))
model.add(layers.Dense(units = 1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model

2022-02-28 16:42:35.496948: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [19]:
# Train the model

history = model.fit(
    X_train, y_train, 
    epochs=10, 
    batch_size=32, 
    validation_split=0.1,
    shuffle=False
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).